In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import warnings

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
x_data = train.drop(['id', 'claim'], axis=1)
y_data = train.claim

x_test = test.drop('id', axis=1)

### data preprocessing

In [ ]:
print('x_data shape {}'.format(x_data.shape))
print('x_test shape {}'.format(x_test.shape))

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score

# model = LogisticRegression()

In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# model = DecisionTreeClassifier()

# from sklearn.model_selection import train_test_split

# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.3, random_state=50)


# model.fit(x_train, y_train)
# y_pred = model.predict_proba(x_val)[:, 1]
# roc_auc_score(y_val, y_pred)

In [ ]:
x_data['n_missing'] = x_data.isna().sum(axis=1)
x_test['n_missing'] = test.isna().sum(axis=1)

In [ ]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy="median")
# x_data = imputer.fit_transform(x_data)
# x_test = imputer.transform(x_test)

In [ ]:
# pd.DataFrame(x_data).isna().sum()
# pd.DataFrame(x_test).isna().sum()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_data_scaler = scaler.fit_transform(x_data)
x_test_scaler = scaler.transform(x_test)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_data_scaler, y_data, test_size=0.3, random_state=50)


### adaboost

In [ ]:
# from sklearn.ensemble import AdaBoostClassifier
# adaboost = AdaBoostClassifier()
# adaboost.fit(x_train, y_train)
# y_pred = adaboost.predict_proba(x_val)[:, 1]

In [ ]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_val, y_pred)

### optuna

In [ ]:
def objective(trial):

    param_grid = {'objective': 'binary:logistic',
              'use_label_encoder': False,
              'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
              'learning_rate': trial.suggest_discrete_uniform('learning_rate',0.01,0.1,0.01),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.3, 1.0, 0.1),
              'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree',0.1,1.0, 0.1),
              'max_depth': trial.suggest_int('max_depth', 2, 20),
              'booster': 'gbtree',
              'gamma': trial.suggest_uniform('gamma',1.0,10.0),
              'reg_alpha': trial.suggest_int('reg_alpha',50,100),
              'reg_lambda': trial.suggest_int('reg_lambda',50,100),
              'random_state': 42,
                 }


    xgb_model = XGBClassifier(**param_grid, tree_method='gpu_hist', predictor='gpu_predictor',
                            eval_metric=['logloss'])

    xgb_model.fit(x_train, y_train, verbose=False)
    y_pred = xgb_model.predict_proba(x_val)[:, 1]
    return roc_auc_score(y_val, y_pred)


In [ ]:
import optuna
from optuna.samplers import TPESampler

train_time = 1 * 30 * 60 # h * m * s
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name='XGBClassifier')
study.optimize(objective, timeout=train_time)

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('\tValue: {}'.format(trial.value))
print('\tParams: ')
for key, value in trial.params.items():
    print('\t\t{}: {}'.format(key, value))

### best params

	Value: 0.8156367214564837
	Params: 
		n_estimators: 2299
		learning_rate: 0.05968414618557474
		subsample: 0.8
		max_depth: 5
		gamma: 1.550895536131489
		reg_alpha: 50
		reg_lambda: 100
		tree_method: gpu_hist
		predictor: gpu_predictor

In [ ]:
xgb_params = trial.params
xgb_params['tree_method'] = 'gpu_hist'
xgb_params['predictor'] = 'gpu_predictor'

# xgb_params = {
#     'objective': 'binary:logistic',
#     'use_label_encoder': False,
#     'n_estimators': 2299,
#     'learning_rate': 0.05968414618557474,
#     'subsample': 0.8,
#     'colsample_bytree': 0.9,
#     'max_depth': 5,
#     'booster': 'gbtree',
#     'gamma': 1.550895536131489,
#     'reg_alpha': 84,
#     'reg_lambda': 93,
#     'tree_method': 'gpu_hist',
#     'predictor': 'gpu_predictor',
#     'n_jobs': 4
# }

# xgb_params = {'objective': 'binary:logistic',
#               'use_label_encoder': False,
#               'n_estimators': 5000,
#               'learning_rate': 0.04,
#               'subsample': 0.66,
#               'colsample_bytree': 0.9,
#               'max_depth': 4,
#               'booster': 'gbtree',
#               'gamma': 3.5,
#               'reg_alpha': 81.8,
#               'reg_lambda': 72.0,
#               'random_state': 42,
#               'tree_method': 'gpu_hist',
#               'predictor': 'gpu_predictor',
#               'n_jobs': 4}

### KFold

In [ ]:
from sklearn.model_selection import KFold

n_split = 10
kfold = KFold(n_split)

val_pred = np.zeros(y_data.shape)
y_test = np.zeros((x_test.shape[0],))

for i, (train_index, val_index) in enumerate(kfold.split(x_data_scaler)):
    # train model
    print("fold {} training".format(i))
    model = XGBClassifier(**xgb_params, eval_metric=['logloss'])
    model.fit(x_data_scaler[train_index], y_data[train_index])
    
    # predict val and test
    val_pred[val_index] = model.predict_proba(x_data_scaler[val_index])[:, 1]
    vla_score = roc_auc_score(y_data[val_index], val_pred[val_index])
    print("fold {} validation auc score {}".format(i, vla_score))
    
    y_test += model.predict_proba(x_test_scaler)[:, 1] / n_split
    
# evaluate validation score    
print("val auc score :", roc_auc_score(y_data, val_pred))

In [ ]:
# xgb_model = XGBClassifier(**xgb_params)

# xgb_model.fit(x_train, y_train)
# y_val_probe = xgb_model.predict_proba(x_val)

In [ ]:
# train_auc = roc_auc_score(y_train, xgb_model.predict_proba(x_train)[:, 1])
# val_auc = roc_auc_score(y_val, y_val_probe[:, 1])

# print("train auc {}\nval auc {}".format(train_auc, val_auc))

train auc 0.8202678946011855
val auc 0.8138542447335897

In [ ]:
# y_test = xgb_model.predict_proba(x_test_scaler)[:, 1]

### submission

In [ ]:
sub_mission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')
sub_mission.claim = y_test
sub_mission.to_csv('submission.csv', index=False)

In [ ]:
import seaborn as sns
sns.histplot(y_test)